### Analysis for Wristband Project

Aug/14/2023


## Import Dependency

In [1]:
import numpy as np
import pandas as pd

## Import Data

Here we use a high dimensional numpy array to keep our data

In [20]:
# subject name to get data path

subject_name = "Hokin"
data_path = "data/" + subject_name
section_numbers = 4

open_data = []
close_data = []
rest_data = []

for i in range (section_numbers):
    section_path = data_path + "/Section_Number_" + str(i)
    open_df = pd.read_csv(section_path + "/open.csv")
    open_data.append(open_df.values)

[array([[ 6.98367023e+00,  6.99119568e+00,  1.00000000e+00, ...,
         1.60658055e-05,  0.00000000e+00,  0.00000000e+00],
       [ 6.98417020e+00,  6.99119568e+00,  1.00000000e+00, ...,
         1.28526453e-05,  0.00000000e+00,  0.00000000e+00],
       [ 6.98467016e+00,  6.99119568e+00,  1.00000000e+00, ...,
         6.42632267e-06,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 7.98615718e+00,  7.98866677e+00,  1.00000000e+00, ...,
         1.92789666e-05,  0.00000000e+00,  0.00000000e+00],
       [ 7.98665714e+00,  7.98866677e+00,  1.00000000e+00, ...,
        -9.63948332e-06,  0.00000000e+00,  0.00000000e+00],
       [ 7.98715687e+00,  7.98866677e+00,  1.00000000e+00, ...,
        -2.57052907e-05,  0.00000000e+00,  0.00000000e+00]])]
[array([[ 6.98367023e+00,  6.99119568e+00,  1.00000000e+00, ...,
         1.60658055e-05,  0.00000000e+00,  0.00000000e+00],
       [ 6.98417020e+00,  6.99119568e+00,  1.00000000e+00, ...,
         1.28526453e-05,  0.00000000e+00,  0.0000000

In [26]:
print(type(open_data[3]))

<class 'numpy.ndarray'>
